In [ ]:
# Imports
import os
import numpy as np
from tifffile import imread, imsave
import xml.etree.ElementTree as ET

In [ ]:
# Import CaImAn and custom functions
import caiman as cm
import utils

In [ ]:
data_folder = '/Users/Henry/Data/temp/Dendrites_Gwen/M3_October_2018'
day_folder = 'M3_2018-10-02'
area_folder = 'S1'
sessions = [
    '09-21-02_Live',
    '09-21-16_Live',
    '09-21-28_Live',
    '09-21-40_Live',
    '09-21-52_Live',
]
sessions = None # either specify selected sessions as list (e.g. ['09-21-02_Live','09-21-16_Live',]) or use None for all sessions
max_sessions = 9999 # restrict number of sessions to process

In [ ]:
n_processes = 4 # number of parallel processes (None to select automatically)

In [ ]:
# create list of sessions to process
if sessions is not None:
    sessions = [os.path.join(data_folder, day_folder, area_folder, x) 
                for (i,x) in enumerate(sessions) if i < max_sessions]
else:
    sessions = sorted([os.path.join(data_folder, day_folder, area_folder, x) 
                       for x in os.listdir(os.path.join(data_folder, day_folder, area_folder)) 
                       if x.endswith('_Live')])[:max_sessions]

In [ ]:
tiff_files = []
xml_files = []
for i_session in sessions:
    tiff_files.append([os.path.join(i_session, x) for x in os.listdir(i_session) if x.endswith('.tif') and not 'stacked' in x][0])
    xml_files.append([os.path.join(i_session, x) for x in os.listdir(i_session) if 'parameters.xml' in x][0])

In [ ]:
# read frame rate from parameters.xml 
for ix, i_session in enumerate(sessions):
    tree = ET.parse(xml_files[ix])
    root = tree.getroot()
    for child in root:
        if child.tag == 'area0': # Note: only area 0!
            fr = child.find('Framerate_Hz')
            frame_rate = float(fr.text)
    if ix <= 10:
        print('Frame rate: %1.4f Hz (%s)' % (frame_rate, os.path.split(i_session)[1]))
if ix > 10:
    print('...')

### Convert mosaic shaped movies to stacked movies

In [ ]:
def mosaic_to_stack(mov, n_planes, x_crop):
    '''
    Convert movie with multiple planes as mosaic (TYX) to ImageJ hyperstack (TZCYXS order)
    mov ... movie as numpy array
    n_planes ... number of planes in Y
    x_crop ... number of pixels to crop in x
    
    Return: stacked movie (ImageJ hyperstack)
    '''
    
    # check if the number of planes matches the shape of the movie
    if mov.shape[1] % n_planes:
        raise Exception('Number of rows in movie must be divisible by number of planes!')
    rows_per_plane = int(mov.shape[1]/n_planes)
    
    # stack planes
    plane_list = []
    for ix in range(n_planes):
        plane_list.append(mov[:,ix*rows_per_plane:(ix+1)*rows_per_plane,:x_crop])
        plane_list[ix] = plane_list[ix][None, ...]
        
    stacked = np.vstack(tuple(plane_list)).swapaxes(0, 1)
    # add dimension for channel
    stacked = stacked[:,:,None,:,:]
    
    return stacked

In [ ]:
# start the cluster (if a cluster already exists terminate it)
if 'dview' in locals():
    dview.terminate()
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=n_processes, single_thread=False)

In [ ]:
%%time
n_planes = 3 # number of planes in movie
x_crop = 0.5 # fraction of x pixels to crop (i.e. 0.5 for half)
for i_file in tiff_files:
    utils.mosaic_to_stack(i_file, n_planes, x_crop)

In [ ]:
%%time
dview.starmap(utils.mosaic_to_stack, [(x, n_planes, x_crop) for x in tiff_files])